<a href="https://colab.research.google.com/github/khanh22mse23057/python4engineer-assignments/blob/main/FinalAssignments_Practice%26Build_API_with_Superstore_Sales_Dataset_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bài tập lớn kết thúc môn

##  *Đặt vấn đề*

Cho tập dữ liệu về doanh số bán hàng gồm các thông tin như sau:
[Input Data](https://drive.google.com/file/d/1QWD8rceAWo6KJYHh6Tx0eBtAM9U1EQno/view?usp=sharing(sad)

[Superstore Sales Dataset Analysis | Kaggle ](https://www.kaggle.com/code/adamossama/superstore-sales-dataset-analysis/data)
- order_id
- order_date
- ship_date
- ship_mode
- customer_id
- customer_name
- segment
- country
- city
- state
- region
- product_id
- sub-category
- product_name
- sales

Hãy sử dụng Python và Jupyter Notebook để trả lời các câu hỏi sau



## Cài đặt môi trường

In [1]:
# Trước hết, bạn cần install Python và package Kaggle
# Một số functions trong bài không work với latest version, nên cài đặt packages version chỉ định như sau
!pip install opendatasets
# !pip install pandas==1.2.0
# !pip install pystan==3.0.0
!pip install pandas
!pip install pystan
!pip install kaggle
!pip install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1319 sha256=d3cb4ef81eae5be6624568ab9eb17d658df6e4c876d66e1b45a094158c27c430
  Stored in directory: /root/.cache/pip/wheels/c5/05/b3/f7b36dbaaf76de31b718cde792c953bfd11d2414a72f204b56
Successfully built unzip



## Chuẩn bị dữ liệu
Để truy cập được tệp data từ Kaggle, cần phải tạo key và cấu hình môi trường

Ví dụ file secret key được tạo từ Kaggle và được lấy về từ github. Cách tạo key : https://prnt.sc/UCwb6zgoPzYT
* Lưu ý: Kaggle yêu cầu xác thực tài khoản bằng số điện thoại để cấp cấp quyền truy cập dữ liệu.

#### Import thư viện

In [ ]:
import os
#Cấu hình secret key để xác thực truy cập đến Kaggle trên môi trường
os.environ['KAGGLE_USERNAME'] = 'khanhphamnguyenphu'
os.environ['KAGGLE_KEY'] = '67b96e181bd01730f8e8ec3cf2b685a7'

!if [ -d ~/.kaggle/ ]; then rm -Rf ~/.kaggle/; fi
!if [ -d ./kaggle/input/ ]; then rm -Rf ./kaggle/input/; fi

!kaggle competitions download -c m5-forecasting-accuracy -p ./kaggle/input/ # Download dữ liệu train từ Kaggle
!unzip  ./kaggle/input/*.zip -d ./kaggle/input/
!ls ./kaggle/input/  #Kiểm tra lại các files đã download


 55% 25.0M/45.8M [00:00<00:00, 50.6MB/s]
100% 45.8M/45.8M [00:00<00:00, 72.4MB/s]
Archive:  ./kaggle/input/m5-forecasting-accuracy.zip
  inflating: ./kaggle/input/calendar.csv  
  inflating: ./kaggle/input/sales_train_evaluation.csv  
  inflating: ./kaggle/input/sales_train_validation.csv  
  inflating: ./kaggle/input/sample_submission.csv  
  inflating: ./kaggle/input/sell_prices.csv  
calendar.csv		     sales_train_evaluation.csv  sample_submission.csv
m5-forecasting-accuracy.zip  sales_train_validation.csv  sell_prices.csv


#### Load dữ liệu

In [3]:
# Đọc dữ liệu từ file csv vào data frame
# calender = pd.read_csv('/content/kaggle/input/calendar.csv')
# train_sales = pd.read_csv('/content/kaggle/input/sales_train_evaluation.csv')
# df_val = pd.read_csv('/content/kaggle/input/sales_train_validation.csv')
# sell_prices = pd.read_csv('/content/kaggle/input/sell_prices.csv')

# # check data completely 
# print('-----------------------------train_sales--------------------------------------')
# train_sales.info()
# print('-----------------------------calender--------------------------------------')
# calender.info()
# print('-----------------------------sell_prices--------------------------------------')
# sell_prices.info()


## Câu 1.Tiền xử lý dữ liệu 
(xử lý các trường hợp thiếu dữ liệu, chuẩn hóa ngày tháng, đơn vị,...).

## Câu 2.Vẽ biểu đồ thể hiện biến động doanh số theo tháng
Cho biết doanh thu trung bình trong 3 tháng gần nhất.


## Câu 3.Vẽ biểu đồ doanh số bán hàng theo quý 


#### Xác định quý nào có tổng doanh số cao nhất

## Câu 4.Tính thời gian trung giao hàng trung bình

## Câu 5.Vẽ biểu đồ thể hiện sức mua theo từng thành phố

#### Xác định top 3 thành phố có doanh số thấp nhất và 3 thành phố có doanh số cao nhất.


## Câu 6.Vẽ biểu đồ thể hiện tỉ trọng doanh số theo bang (state)

#### Xác định trung bình doanh số của 5 bang dẫn đầu về sức mua.

## Câu 7.Cho biết các bang dẫn đầu doanh số theo từng vùng
(east, west, south, central,..)

## Câu 8.Xác định top 3 sản phẩm có doanh thu cao nhất và top 3 sản phẩm có số lượng đơn vị bán ra cao nhất

## Câu 9.Vẽ biểu đồ thể hiện tỉ trọng doanh số của từng category

## Câu 10.Dự báo doanh số theo nhiều khía cạnh khác nhau trong tương lai: 
theo vùng, theo bang, theo category

# Triển khai API